In [1]:
import numpy as np
import copy
import os
import unittest

from dftpy.constants import ENERGY_CONV
from dftpy.formats import io

from edftpy.utils.common import Field, Grid, Atoms
from edftpy.functional import LocalPP, KEDF, Hartree, XC
from edftpy.optimizer import Optimization
from edftpy.evaluator import EmbedEvaluator, EvaluatorOF, TotalEvaluator
from edftpy.enginer.of_dftpy import DFTpyOF
from edftpy.density.init_density import AtomicDensity
from edftpy.subsystem.subcell import SubCell, GlobalCell
from edftpy.mixer import PulayMixer, LinearMixer
from edftpy.mpi import GraphTopo, MP

In [2]:
def get_optimizer(ke_kwargs, xc_kwargs = {}, method = 'full'):
    data_path = os.environ.get('EDFTPY_DATA_PATH')
    if not data_path : data_path = 'DATA/'
    if not os.path.exists(data_path) : data_path = '../DATA/'
    data_path += '/'
    path_pp = data_path
    path_pos = data_path

    pp_al ='Al_OEPP_lda.recpot'
    posfile='fcc.vasp'

    ions = io.read(path_pos+posfile, names=['Al'])
    gsystem = GlobalCell(ions, grid = None, ecut = 22.05, full = False, optfft = True)
    grid = gsystem.grid
    ############################## Functionals  ##############################
    pplist = {'Al': path_pp+pp_al}
    pseudo = LocalPP(grid = grid, ions=ions,PP_list=pplist,PME=True)
    hartree = Hartree()
    xc = XC(**xc_kwargs)
    emb_ke_kwargs = {'name' :'TF'}
    ke = KEDF(**emb_ke_kwargs)
    funcdicts = {'KE' :ke, 'XC' :xc, 'HARTREE' :hartree, 'PSEUDO' :pseudo}
    total_evaluator = TotalEvaluator(**funcdicts)
    #-----------------------------------------------------------------------
    gsystem.total_evaluator = total_evaluator
    graphtopo = GraphTopo()
    mp = MP(comm = graphtopo.comm_sub)
    gsystem.graphtopo = graphtopo
    #-----------------------------------------------------------------------
    index_a = None
    atomicd = AtomicDensity()
    driver_a = gen_sub_of(ions, grid, pplist, index_a, atomicd, xc_kwargs, ke_kwargs, emb_ke_kwargs = emb_ke_kwargs, gsystem = gsystem, method = method, mp = mp)
    drivers = [driver_a]
    graphtopo.build_region(grid=gsystem.grid, drivers=drivers)
    return gsystem, drivers

In [3]:
def gen_sub_of(ions, grid, pplist = None, index = None, atomicd = None, xc_kwargs = {}, ke_kwargs = {}, emb_ke_kwargs = {}, gsystem = None, method = 'full', mp = None, **kwargs):
    if atomicd is None :
        atomicd = AtomicDensity()
    mixer = PulayMixer(predtype = 'kerker', predcoef = [0.8, 1.0], maxm = 7, coef = 0.8, predecut = 0, delay = 1)
    #-----------------------------------------------------------------------
    if ke_kwargs is None or len(ke_kwargs) == 0 :
        ke_evaluator = None
    else :
        ke_evaluator = KEDF(**ke_kwargs)

    ke_emb_a = KEDF(**emb_ke_kwargs)
    emb_funcdicts = {'KE' :ke_emb_a}

    ke_sub_kwargs = {'name' :'vW'}
    ke_sub = KEDF(**ke_sub_kwargs)

    sub_funcdicts = {}
    sub_funcdicts['KE'] = ke_sub
    evaluator_of = EvaluatorOF(gsystem = gsystem, **sub_funcdicts)
    embed_evaluator = EmbedEvaluator(ke_evaluator = ke_evaluator, **emb_funcdicts)
    #-----------------------------------------------------------------------
    subsys_a = SubCell(ions, grid, index = index, cellcut = [0.0, 0.0, 10.5], optfft = True, mp = mp)
    ions_a = subsys_a.ions
    rho_a = subsys_a.density
    rho_a[:] = atomicd.guess_rho(ions_a, subsys_a.grid)
    options = {"method" :'CG-HS', "maxiter": 220, "econv": 1.0e-6, "ncheck": 2, "opt_method" : method}
    of_enginer_a = DFTpyOF(evaluator = embed_evaluator, mixer = mixer, options = options, subcell = subsys_a, evaluator_of = evaluator_of)
    return of_enginer_a

In [4]:
def get_energy(gsystem, drivers):
    optimization_options = {'econv' : 1e-6, 'maxiter' : 70}
    optimization_options["econv"] *= gsystem.ions.nat
    opt = Optimization(drivers = drivers, options = optimization_options)
    opt.optimize(gsystem = gsystem)
    energy = opt.energy
    opt.stop_run()
    return energy

In [5]:
xc_kwargs = {"x_str":'lda_x','c_str':'lda_c_pz', 'libxc' :False}
ke_kwargs = {'name' :'TF'}
gsystem, drivers =get_optimizer(ke_kwargs, xc_kwargs = xc_kwargs)
energy = get_energy(gsystem, drivers)
print('TFvW : energy', energy) # -8.281114354275829

GlobalCell grid [16 16 16]
setting key: Al -> /home/sxc/work/edftpy/eDFTpy/examples/DATA//Al_OEPP_lda.recpot
Subcell grid(sub_of): [16 16 16]  [16 16 16]
Subcell shift(sub_of): [0 0 0]

Begin optimize
Optimization options :
{'econv': 4e-06,
 'maxiter': 70,
 'ncheck': 2,
 'olevel': 2,
 'pconv': 4e-08,
 'pconv_sub': array([4.e-08]),
 'sdft': 'sdft'}
update density
density 12.00000000000027
          Step    Energy(a.u.)            dE              dP        dC        Time(s)         
econv 1 0.01
res_norm(sub_of): 1  0.004461428594984468  0.0035481626726263397
mixing parameters :  0.8
delay : use output density
Norm of reidual density : 
[0.00354816]
Energy of reidual density : 
[0.00696973]
sub_energy(sub_of): 1  5.319724870166145
----------------------------------------------------------------------------------------------------
   Embed: 0       -3.248045197108E-01     -3.248045E-01   6.97E-03  6.97E-03  7.635736E-02    
-----------------------------------------------------------------

# Simply use a input file with parameters

In [6]:
from edftpy.interface import optimize_density_conf, conf2init, conf2output, config2optimizer
from edftpy.config import read_conf

In [7]:
input_str="""
[JOB]
task            = Optdensity

[PATH]
pp              = ../DATA/
cell            = ../DATA/

[PP]
Al              = Al_OEPP_lda.recpot

[OPT]
maxiter         = 100
econv           = 1e-6

[GSYSTEM]
cell-file       = fcc.vasp
grid-ecut       = 1000
exc-x_str       = lda_x
exc-c_str       = lda_c_pz
kedf-kedf       = TF

[SUB_OF]
calculator      = dftpy
embed           = KE
cell-index      = :
kedf-kedf       = TFvW
density-initial = heg
"""

In [8]:
config=read_conf(input_str)
graphtopo=conf2init(config,parallel=False)

Serial version on        1 processor


In [9]:
optimizer = config2optimizer(config, graphtopo = graphtopo)

Communicators recreated :  1
Number of subsystems :  1
Number of processors for each subsystem : 
  [1]
GlobalCell grid [20 20 20]
setting key: Al -> ../DATA//Al_OEPP_lda.recpot
Subcell grid(sub_of): [20 20 20]  [20 20 20]
Subcell shift(sub_of): [0 0 0]



In [10]:
optimizer.optimize()

Begin optimize
Optimization options :
{'econv': 4e-06,
 'maxiter': 100,
 'method': 'Normal',
 'ncheck': 2,
 'olevel': 2,
 'pconv': 4e-08,
 'pconv_sub': array([4.e-08]),
 'sdft': 'sdft'}
update density
density 11.999999999999018
          Step    Energy(a.u.)            dE              dP        dC        Time(s)         
econv 1 0.04
res_norm(sub_of): 1  0.00449936830663971  0.0034228953698076012
Norm of reidual density : 
[0.0034229]
Energy of reidual density : 
[0.00614854]
sub_energy(sub_of): 1  5.316233057838957
----------------------------------------------------------------------------------------------------
   Embed: 0       -3.282963345008E-01     -3.282963E-01   6.15E-03  6.15E-03  1.126130E-01    
----------------------------------------------------------------------------------------------------
econv 2 0.004
res_norm(sub_of): 2  0.002170139437163165  0.0005303712343241861
Norm of reidual density : 
[0.00053037]
Energy of reidual density : 
[0.00012317]
sub_energy(sub_of): 

In [11]:
# remove the temporary files
import os
files=['sub_of.vasp','sub_of.out', 'edftpy_cell.vasp', 'edftpy_running.json']
for f in files:
    if os.path.isfile(f): os.remove(f)